In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVR

input_df = pd.read_csv("modifiedIndustriesPerCountyGA-2012.csv")
input_df = input_df.drop(['Column'], axis=1)

output_df = pd.read_csv("incidencerates.csv")
output_df = output_df[['locale', 'fips', 'annual_count_avg']]  # 'trend_last_5' missing data in 10 counties

In [2]:
input_df.head()

,GEO_TTL,COUNTY,YEAR,NAICS2012,NAICS2012_TTL,ESTAB,EMP,PAYANN,state,county
0,"Appling County, Georgia",1,2012,113,Forestry and logging,11,61,2256,13,1
1,"Appling County, Georgia",1,2012,115,Support activities for agriculture and forestry,1,0,0,13,1
2,"Appling County, Georgia",1,2012,221,Utilities,7,0,0,13,1
3,"Appling County, Georgia",1,2012,236,Construction of buildings,12,113,2572,13,1
4,"Appling County, Georgia",1,2012,237,Heavy and civil engineering construction,2,0,0,13,1


In [3]:
output_df.head()

,locale,fips,annual_count_avg
0,"Pickens County(7,8)",227,249
1,"Towns County(7,8)",281,116
2,"Union County(7,8)",291,226
3,"Lee County(7,8)",177,154
4,"Turner County(7,8)",287,56


In [4]:
industry_keys = {}

for naics in input_df['NAICS2012']:
    if naics not in industry_keys:
        industry_keys[naics] = 0

# industry_keys['year'] = 2012
industry_keys['counties'] = []
industry_keys['annual_count_avg'] = 0

for county in output_df['fips']:
    industry_keys['counties'].append(county)

In [6]:
# industry_keys

In [17]:
df = pd.DataFrame(industry_keys, index=industry_keys['counties'])
df.set_index('counties')

for idx, row in input_df.iterrows():
#     df.set_value(row['COUNTY'], row['NAICS2012'], row['ESTAB'])
    df.loc[row['COUNTY'], row['NAICS2012']] = row['ESTAB']

for idx, row in output_df.iterrows():
#     df.set_value(row['fips'], 'annual_count_avg', row['annual_count_avg'])
    df.loc[row['fips'], 'annual_count_avg'] = row['annual_count_avg']

In [18]:
df.head()

,113,115,221,236,237,238,321,325,327,331,...,525,533,114,483,316,487,211,521,counties,annual_count_avg
227,4,1,3,18,10,41,2,1,4,0,...,0,1,0,0,0,0,0,0,227,249
281,0,0,1,10,4,23,0,0,1,0,...,0,0,0,0,0,0,0,0,281,116
291,1,1,3,23,9,29,2,0,1,0,...,0,0,0,0,0,0,0,0,291,226
177,1,4,1,8,3,47,3,0,0,0,...,0,0,1,0,0,0,0,0,177,154
287,1,1,1,1,1,3,2,1,2,0,...,0,0,0,0,0,0,0,0,287,56


In [20]:
X = df.loc[:, :'counties']
y = df['annual_count_avg']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [23]:
X.head()

,113,115,221,236,237,238,321,325,327,331,...,334,525,533,114,483,316,487,211,521,counties
227,4,1,3,18,10,41,2,1,4,0,...,0,0,1,0,0,0,0,0,0,227
281,0,0,1,10,4,23,0,0,1,0,...,0,0,0,0,0,0,0,0,0,281
291,1,1,3,23,9,29,2,0,1,0,...,0,0,0,0,0,0,0,0,0,291
177,1,4,1,8,3,47,3,0,0,0,...,0,0,0,1,0,0,0,0,0,177
287,1,1,1,1,1,3,2,1,2,0,...,0,0,0,0,0,0,0,0,0,287


In [25]:
y.head()

227    249
281    116
291    226
177    154
287     56
Name: annual_count_avg, dtype: int64

In [26]:
svr = LinearSVR(random_state=0, tol=1e-5).fit(X_train, y_train)
print(svr.score(X_test, y_test))

# Test single prediction for county 307
pred = np.array(df.loc[307, :'counties']).reshape(1, -1)
svr.predict(pred)

0.9667265165816036


/Users/merrelbook/anaconda3/envs/datalab/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


array([-1.55728959])